# ParabolaTree
Extra required packages:
- matplotlib

In [ ]:
import gurobipy as gp
import numpy as np
from gurobipy import GRB
from matplotlib import pyplot as plt
from sklearn import model_selection
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import make_pipeline

from gurobi_ml import add_predictor_constr

In [ ]:
x = np.arange(-1, 1, 0.005)

z = (x) ** 2

In [ ]:
plt.plot(x, z)

plt.show()

In [ ]:
X = np.concatenate([x.ravel().reshape(-1, 1)], axis=1)
Y = z.ravel()

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, Y)

In [ ]:
# Run our regression
layers = [56, 56]
regression = MLPRegressor(hidden_layer_sizes=layers, activation="relu")
regression = make_pipeline(regression)
regression.fit(X=X_train, y=y_train)

In [ ]:
plt.plot(X, regression.predict(X))

plt.show()

### Do the optimization model

In [ ]:
npoints = 1
optfeat = [0]

In [ ]:
# Start with classical part of the model
m = gp.Model()

x = m.addMVar((npoints, len(optfeat)), lb=-1, ub=1, name="x")
y = m.addMVar((npoints, 1), lb=-GRB.INFINITY, name="y")

m.setObjective(y.sum(), gp.GRB.MINIMIZE)

# Add constraint to predict value of y using kwnown and to compute features
pconstr = add_predictor_constr(m, regression, x, y)

### Finally optimize it

In [ ]:
m.Params.TimeLimit = 10
m.Params.MIPGap = 0.01

In [ ]:
m.optimize()

In [ ]:
print(f"Maximal error in predicted values in solution {np.max(np.abs(pconstr.get_error()))}")

### Look at the solution

In [ ]:
x.X

Copyright © 2022 Gurobi Optimization, LLC